# Electricity prediction

Source: https://medium.com/@kylejones_47003/neural-networks-for-time-series-with-tensorflow-keras-in-python-0479918268f1

In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio # settings like template by default
pio.templates.default = 'plotly_dark' # default template

In [8]:
df = pd.read_csv("data/energy.csv", usecols=["timestamp", "load"])
df.timestamp = pd.to_datetime(df.timestamp)
df.sort_values(by=['timestamp'], inplace=True, ascending=True)
df.set_index('timestamp', inplace=True)
print(f"Data {df.shape}")
df.head(2)

Data (26304, 1)


,load
timestamp,
2012-01-01 00:00:00,2698.0
2012-01-01 01:00:00,2558.0


In [45]:
# Create lagged features
def create_features(data, lag=3):
    X, y = [], []
    for i in range(len(data) - lag):
        X.append(data[i:i + lag])
        y.append(data[i + lag])
    X = np.array(X)
    X = X.reshape((X.shape[0], lag))
    y = np.array(y)
    return X, y

lag = 12
X, y = create_features(df.values, lag=lag)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Train labels shape: {y_train.shape}, Test labels shape: {y_test.shape}")

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a simple feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(lag,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=False, validation_split=0.1)

Train shape: (21033, 12), Test shape: (5259, 12)
Train labels shape: (21033, 1), Test labels shape: (5259, 1)


In [46]:
# Evaluate and predict
y_pred = model.predict(X_test)
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print("RMSE:", rmse)

df_test = df.tail(X_test.shape[0]).copy()
df_test['y_pred'] = y_pred.flatten()

fig = px.line(df_test, x = df_test.index, y= ['load', 'y_pred'],
            labels={'value': 'Energy', 'variable': 'Values'},
            title=f'Feedforward Neural Network Forecast RMSE: {rmse:.3f}')
fig.show()

165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
RMSE: 67.54268063508624
